# Database manager
1. Join multiple csv
2. Remove the password from the match
2. Shuffle the csv
3. Split in _train_ and _eval_ databases ready for ML

## Params
- **PLAYER_ALGORITHM**
    - The database that use for the process
- **ONLY_TRAIN:** 
    - if _True_ = the system only join all data inside "dbPlayers/_player_/chest/*.csv"
    - if _False_ = the system join all data inside "dbPlayers/_player_/chest/*.csv", then remove password, split in train and eval and save everything inside "dbML/_player_/*_train.csv" and "dbML/_player_/*_wval.csv"
- **TRAIN_QUOTE:**
    - If _ONLY_TRAIN=False_ this parameter specify the quote for split the db in Train/Eval

In [15]:
PLAYER_ALGORITHM = 'hopeful' # [knuth, knuthFast, hopeful]
ONLY_JOIN = False
TRAIN_QUOTE = 0.8

-----------------
**Environment prepare**

In [16]:
from datetime import datetime
import os
DATE = datetime.today().strftime('%Y-%m-%d')
os.putenv("PLAYER_ALGORITHM", PLAYER_ALGORITHM)
PATHS_CSV_TO_JOIN = !find ./dbPlayers/${"PLAYER_ALGORITHM"}/chest/ | grep csv | grep -v _cuts
OUT_FOLDER = 'dbPlayers' if ONLY_JOIN else 'dbML'
OUTPUT_PATH = './{out_folder}/{player}/{player}_{date}'.format(out_folder=OUT_FOLDER, 
                                                               player=PLAYER_ALGORITHM, 
                                                               date=DATE)
print('OUTPUT_PATH(prefix):',OUTPUT_PATH)

OUTPUT_PATH(prefix): ./dbML/hopeful/hopeful_2019-10-05


------------------------
## Data pipeline
### 1. Load and remove the psw from the match

In [17]:
import pandas as pd
import numpy as np

In [18]:
def replace_last_guess(psw):
    if psw[-2] == '4':
        return '<pad>'
    else:
        return psw

In [19]:
# Params
df_list = []
features_name = ['Guess 1', 'Guess 2', 'Guess 3', 'Guess 4', 'Guess 5',
                  'Guess 6', 'Guess 7', 'Guess 8', 'Guess 9', 'Guess 10']
target_name = 'PASSWORD'

# Optimize the df load
feature_type = {}
feature_type[target_name] = str
for feature in features_name:
    feature_type[feature] = str

# Load the db
for path in PATHS_CSV_TO_JOIN:
    df = pd.read_csv(path, delimiter=',',encoding='utf-8', skip_blank_lines=True, dtype=feature_type)
    if not ONLY_JOIN:
        for feature in df.keys():
            df[feature] = df[feature].map(replace_last_guess)
    df_list.append(df)

# Join the csv
df_total = pd.concat(df_list, sort = False)
print('New dataframe len:{}'.format(len(df_total)))
df_total.iloc[:2]

New dataframe len:1036800


,Guess 1,Guess 2,Guess 3,Guess 4,Guess 5,Guess 6,Guess 7,Guess 8,Guess 9,Guess 10,PASSWORD
0,ADDB10,ACFC10,ECBB00,FDFF00,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,AAAA
1,BEAD11,FEDC00,ABAA22,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,AAAB


### 2. Shuffle and split

In [20]:
# Shuffle for train - Eval
if not ONLY_JOIN:
    df_total = df_total.sample(frac=1, random_state=42)
    msk = np.random.rand(len(df_total)) < TRAIN_QUOTE
    df_train = df_total[msk]
    df_eval = df_total[~msk]
    print('[INFO] Train size:\t{}\n[INFO] Eval size:\t{}'.format(len(df_train),len(df_eval)))
    df_train.iloc[:2]

[INFO] Train size:	829843
[INFO] Eval size:	206957


### 3. Saving

In [21]:
if ONLY_JOIN:   
    # Match history
    with open(OUTPUT_PATH+".csv", 'w') as f:
        df_total.to_csv(f, sep=',', encoding='utf-8',index=False, header=True)
    print('Joined file saved')
    
    # Cuts histories
    df_list_cut = []
    for path in PATHS_CSV_TO_JOIN:
        path = path[:-4] + '_cuts.csv'
        df_cut = pd.read_csv(path, delimiter=',',encoding='utf-8', skip_blank_lines=True)
        df_cut[features_name] = df_cut[features_name].apply(pd.to_numeric, downcast='integer')
        df_list_cut.append(df_cut)
    df_total_cuts = pd.concat(df_list_cut, sort = False)
    
    with open(OUTPUT_PATH+"_cuts.csv", 'w') as f:
        df_total_cuts.to_csv(f, sep=',', encoding='utf-8',index=False, header=True)
    print('Cuts file saved')
    

else:
    # Train ML database
    with open(OUTPUT_PATH+"_{}_{}_train.csv".format(TRAIN_QUOTE, len(df_train)), 'w') as f:
        df_train.to_csv(f, sep=',', encoding='utf-8',index=False, header=True)
    
    # Eval ML database
    with open(OUTPUT_PATH+"_{}_{}_eval.csv".format(TRAIN_QUOTE, len(df_eval)), 'w') as f:
        df_eval.to_csv(f, sep=',', encoding='utf-8',index=False, header=True)

    print('Train and eval saved')

Train and eval saved
